# This notebook is to downlod user tweets

In [73]:
from twitter import *

In [74]:
import pandas as pd
import json
from os.path import join
from time import time
from tqdm import tqdm_notebook as tqdm

import csv
import numpy as np

In [85]:
key = "Xp753Z1aLbulkjsyNcyb5zhD5"
secret = "nzQOmJaIMzC6HhCkFOl7GOxikIlQrmKW2CPEkAHWQZzu0M69dL"
token="796315603959246848-9wEDy2sMEBcQh07NfZ3OwCcvOii6M6D"

In [86]:
twitter = Twitter(auth = OAuth("796315603959246848-9wEDy2sMEBcQh07NfZ3OwCcvOii6M6D",
                  "vH7MdLICCc6tJvrjgESbCCpxXOApQ64gqGSajqvtF8YeG",
                  "Xp753Z1aLbulkjsyNcyb5zhD5",
                  "nzQOmJaIMzC6HhCkFOl7GOxikIlQrmKW2CPEkAHWQZzu0M69dL"))

In [87]:
user_tweets_map = {}
twitter_errors = []
total_request_made = 0
total_request_limit = 90000

In [88]:
def get_all_tweets(user_id, total_request_made):
    total_results = []
    results = twitter.statuses.user_timeline(user_id = user_id, count = 200)
    total_results.extend(results)
    if len(results) == 0:
        return total_results
    
    last_id = results[-1]['id']

    while last_id != None:
        try:
            results = twitter.statuses.user_timeline(user_id = user_id, count = 200, max_id = last_id)
            total_request_made += 1
        except TwitterHTTPError as error:
            twitter_errors.append(error)
        
        if len(results) == 0:
            last_id = None
        elif last_id == results[-1]['id']:
            last_id = None
        else:
            last_id = results[-1]['id']
            total_results.extend(results)
    
    return total_results

In [89]:
genders=['male','female']

In [90]:
def get_user_tweets():
    t0 = time()
    csv_path = "C:\\Users\\pvashisth\\Desktop\\ai_course\\testing\\data.csv"
    
    df = pd.read_csv(csv_path, encoding='latin-1')
    
    df = df.loc[(df['gender'].isin(genders)) & (df['gender:confidence'] == 1)]
    print(df.shape)
    
    for index, row in tqdm(df.iterrows()):
        if total_request_made >= total_request_limit:
            print("Stopping for today!!!!")
            print("Next user to be processed is {}: ".format(row['user_id']))
            return
        try:
            results = get_all_tweets(row['user_id'], total_request_made)
        except TwitterHTTPError as error:
            twitter_errors.append(error)
        
        user_tweets_map[row['user_id']] = {
            'gender': row['gender'],
            'results': results
        }
    
    # save the dict.
    with open(join('C:\\Users\\pvashisth\\Desktop\\ai_course\\testing', 'user_all_tweets_data_part_1'), 'w', encoding='utf-8') as w:
        json.dump(user_tweets_map, w)
    
    print("Completed in {}".format(time() - t0))

In [91]:
get_user_tweets()

(10020, 26)


Completed in 3792.848995447159


In [38]:
len(user_tweets_map)

10020

In [39]:
type(user_tweets_map)

dict

In [40]:
total_tweets = []
for utm in user_tweets_map:
    total_tweets.append(len(user_tweets_map[utm]['results']))

In [41]:
len(total_tweets)

10020

In [42]:
np.mean(total_tweets), np.median(total_tweets), np.std(total_tweets), np.min(total_tweets), np.max(total_tweets)

(21.567964071856288, 8.0, 53.261729674793735, 0, 200)

## Data transformation - creating dataset to train ML model

In [3]:
user_tweets_map = {}

In [4]:
with open("C:\\jupyter\\user_tweets_data", 'r') as r:
    user_tweets_map = json.loads(r.read())

In [5]:
len(user_tweets_map)

10020

In [26]:
len(user_tweets_map['815719226']['results'][0]['text'].split())

14

In [27]:
tweets_lengths = []
for user_id in user_tweets_map:
    tweets_data = user_tweets_map[user_id]['results']
    
    for tweet in tweets_data:
        tweets_lengths.append(len(tweet['text'].split()))

In [28]:
len(tweets_lengths)

216111

In [29]:
np.mean(tweets_lengths), np.median(tweets_lengths), np.std(tweets_lengths), np.min(tweets_lengths), np.max(tweets_lengths)

(7.130631943769637, 3.0, 6.906577762115752, 1, 59)

In [30]:
more_than_50 = [t for t in tweets_lengths if t >=50]
more_than_40 = [t for t in tweets_lengths if t >=40]
more_than_30 = [t for t in tweets_lengths if t >=30]
more_than_20 = [t for t in tweets_lengths if t >=20]
more_than_10 = [t for t in tweets_lengths if t >=10]
more_than_5 = [t for t in tweets_lengths if t >=5]

len(more_than_50), len(more_than_40), len(more_than_30), len(more_than_20), len(more_than_10), len(more_than_5)

(1, 1, 142, 17361, 61731, 96523)

## END

In [41]:
for status in results:
    print("(%s) %s" % (status["created_at"], status["text"].encode("ascii", "ignore")))

(Tue Sep 03 17:04:23 +0000 2019) b'CAMP MARS MALIBU THIS WEEKEND!!! Whos coming to celebrate 10 years of THIS IS WAR with us?  https://t.co/O1r4Na2T2h'
(Sun Sep 01 16:00:01 +0000 2019) b' https://t.co/bjPa2TjDjO'
(Sat Aug 31 16:05:36 +0000 2019) b' https://t.co/QYafM7GBr9'
(Sat Aug 31 01:18:34 +0000 2019) b' https://t.co/EuOkech6xx'
(Wed Aug 28 17:21:52 +0000 2019) b'  https://t.co/ju5htWnAaW'
(Tue Aug 27 16:25:32 +0000 2019) b'Someone on Mars loves me\n\nhttps://t.co/JZ1ivXCX1Y https://t.co/Qs4BmhYLJY'
(Mon Aug 26 20:54:38 +0000 2019) b'Pre-show ritual https://t.co/0uXff8dXl2'
(Mon Aug 26 16:51:03 +0000 2019) b' https://t.co/CQDcmBtK5C'
(Sat Aug 24 17:27:58 +0000 2019) b'\n\n.@GQSpain \n@gucci \nAlessandro Michele\n Cameron McCool https://t.co/xrq8HmoZ0P'
(Fri Aug 23 16:31:49 +0000 2019) b"Who's a good boy? \n\n Alessandro Michele https://t.co/0WKnuY6OAQ"
(Thu Aug 22 23:00:43 +0000 2019) b"It's been two years since #WalkOnWater came out. years of sharing and singing this song all over

In [ ]:
# run

In [9]:
import csv
import pandas as pd
import numpy as np

In [15]:
data=pd.read_csv('data.csv',encoding="latin-1")

In [23]:

data.describe()

,user_id,_trusted_judgments,gender:confidence,profile_yn:confidence,fav_number,retweet_count,tweet_count,tweet_id
count,2.005000e+04,20050.000000,20024.000000,20050.000000,20050.000000,20050.000000,2.005000e+04,2.005000e+04
mean,8.157294e+08,3.615711,0.882756,0.993221,4382.201646,0.079401,3.892469e+04,6.590000e+17
std,6.000801e+03,12.331890,0.191403,0.047168,12518.575919,2.649751,1.168371e+05,1.536038e+05
min,8.157192e+08,3.000000,0.000000,0.627200,0.000000,0.000000,1.000000e+00,6.590000e+17
25%,8.157243e+08,3.000000,0.677800,1.000000,11.000000,0.000000,2.398000e+03,6.590000e+17
50%,8.157294e+08,3.000000,1.000000,1.000000,456.000000,0.000000,1.144150e+04,6.590000e+17
75%,8.157345e+08,3.000000,1.000000,1.000000,3315.500000,0.000000,4.002750e+04,6.590000e+17
max,8.157580e+08,274.000000,1.000000,1.000000,341621.000000,330.000000,2.680199e+06,6.590000e+17
